# Data Manipulation in SQL

# 1. CASE

## CASE statements


    SELECT *
    FROM table
    WHERE 
        CASE WHEN a > 5 THEN 'Keep'
            WHEN a <= 5 THEN 'Exclude' END = 'Keep';


    SELECT 
        CASE WHEN hometeam_id = 10189 THEN 'FC Schalke 04'
            WHEN hometeam_id = 9823 THEN 'FC Bayern Munich'
            ELSE 'Other' END AS home_team,
        COUNT(id) AS total_matches
    FROM matches_germany
    GROUP BY home_team;


    SELECT 
        season,
        date,
        home_goal,
        away_goal
    FROM matches_italy
    WHERE 
        -- Find games where home_goal is more than away_goal
        CASE WHEN hometeam_id = 9857 AND home_goal > away_goal THEN 'Bologna Win'
            -- Find games where away_goal is more than home_goal
            WHEN awayteam_id = 9857 AND away_goal > home_goal THEN 'Bologna Win' 
            -- Exclude games not won by Bologna
            END IS NOT NULL;

## CASE WHEN with aggregate functions


###  CASE WHEN with COUNT
a number, a string of text, or any column in the table, SQL is COUNTing the number of rows returned by the CASE statement.

    SELECT 
        c.name AS country,
        -- Count matches in 2012/2013
        COUNT(CASE WHEN m.season = '2012/2013' THEN m.id END) AS matches_2012_2013,
        -- Count matches in 2013/2014
        COUNT(CASE WHEN m.season = '2013/2014' THEN m.id END) AS  matches_2013_2014
    FROM country AS c
    LEFT JOIN match AS m
    ON c.id = m.country_id
    GROUP BY country;


### CASE WHEN with SUM 
the SUM function to calculate a total of any value

    SELECT 
    c.name AS country,
    SUM(CASE WHEN m.season = '2012/2013' AND m.home_goal > m.away_goal THEN 1 ELSE 0 END) AS matches_2012_2013,
    SUM(CASE WHEN m.season = '2013/2014' AND m.home_goal > m.away_goal THEN 1 ELSE 0 END) AS matches_2013_2014,
    SUM(CASE WHEN m.season = '2014/2015' AND m.home_goal > m.away_goal THEN 1 ELSE 0 END) AS matches_2014_2015
    FROM country AS c
    LEFT JOIN match AS m ON c.id = m.country_id
    GROUP BY country;


## CASE WHEN with AVG
calculate an average of data

    SELECT 
        c.name AS country,
        -- Calculate the fraction of tied games in each season
        AVG(CASE WHEN m.season='2013/2014' AND m.home_goal = m.away_goal THEN 1
                WHEN m.season='2013/2014' AND m.home_goal != m.away_goal THEN 0
                END) AS ties_2013_2014,
        AVG(CASE WHEN m.season='2014/2015' AND m.home_goal = m.away_goal THEN 1
                WHEN m.season='2014/2015' AND m.home_goal != m.away_goal THEN 0
                END) AS ties_2014_2015
    FROM country AS c
    LEFT JOIN matches AS m
    ON c.id = m.country_id
    GROUP BY country;


## Percentages with CASE and AVG



# 2. Subqueries

A subquery can be placed in any part of your query -- such as the SELECT, FROM, WHERE, or GROUP BY clause

## Simple subqueries

A simple subquery is a query, nested inside another query, that can be run on its own

### Subqueries in the WHERE clause
for filtering results based on information you'd have to calculate separately

    SELECT 
        date,
        home_goal,
        away_goal
    FROM matches_2013_2014
    -- Filter for matches where total goals is greater than 3x the average
    WHERE (home_goal + away_goal) >  
        (SELECT 3 * AVG(home_goal + away_goal)
            FROM matches_2013_2014); 


### Subqueries filtering list with IN
generating a filtering list

    SELECT 
        -- Select the team long and short names
        team_long_name,
        team_short_name
    FROM team 
    -- Exclude all values from the subquery
    WHERE team_api_id NOT IN
        (SELECT DISTINCT hometeam_id  FROM match);

### Subqueries in FROM
transform your data into a different shape, or pre-filter it before making calculations. Subqueries in a FROM statement are a common way of preparing that data. Subqueries in FROM are also useful when calculating aggregates of aggregate information. 